In [1]:
from sifce import datatools, sims, network, tv
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical
from astropy.cosmology import Planck18
pd.set_option('display.max_columns', None)

In [2]:
net = network.Network(ifo_uptime_file="segment_list.segs",ifos=["H1", "L1", "V1"], psd_config_file="net_psds.psds")
net.load_psds()
net.psds_for_time(5)

{'H1': <pycbc.types.frequencyseries.FrequencySeries at 0x7f943dcaa9e0>,
 'L1': <pycbc.types.frequencyseries.FrequencySeries at 0x7f943dcaaa40>,
 'V1': <pycbc.types.frequencyseries.FrequencySeries at 0x7f90b9d8d390>}

In [3]:
prior_gw150914 = dict(mass_1= DeltaFunction(35),
                     mass_2= DeltaFunction(35),
                     a_1 = DeltaFunction(0.5),
                     a_2 = DeltaFunction(0.5),
                     tilt_1 = DeltaFunction(1.4),
                     tilt_2 = DeltaFunction(1.4),
                     phi_12 = DeltaFunction(3.09),
                     phi_jl = DeltaFunction(5.21),
                     theta_jn =  Sine(name='theta_jn'),
                     phase = DeltaFunction(1.04),
                     reference_frequency = DeltaFunction(20),
                     dec=Cosine(name='dec'),
                     ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                     psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),
                     geocent_time = DeltaFunction(5),
                     luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000, unit='Mpc', latex_label='$d_L$'))

In [4]:
prior_obj = PriorDict(dictionary = prior_gw150914)
pop = sims.SimulationSet(distribution=prior_obj, cosmology_model=Planck18, network=net)
pop.sample_distribution(100)

In [5]:
np.median(pop.simulations_dataframe["opt_snr_net"])

nan

In [6]:
import copy
list_rows_sky = list()
for index, row in pop.simulations_dataframe.iterrows():
    row = copy.copy(row)
    row["luminosity_distance"]=1
    list_rows_sky.append(pop.calc_snrs_sky(row, N=10, benchmark=True))

In [7]:
pop.simulations_dataframe = pd.concat(list_rows_sky, ignore_index=True)

In [8]:
np.median(pop.simulations_dataframe["opt_snr_net"])

1.0735769118393053

In [9]:
pop.simulations_dataframe[pop.simulations_dataframe["opt_snr_net"] >= 2]

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,intrinsic_index,reference_snr_index,cal_idx,opt_snr_H1,mf_snr_H1,opt_snr_L1,mf_snr_L1,opt_snr_V1,mf_snr_V1,opt_snr_net,mf_snr_net,iota,phi_1,phi_2


In [10]:
list_rows_distance = list()
for index, row in pop.simulations_dataframe.iterrows():
    list_rows_distance.append(pop.sample_distances(row, N=10))

In [11]:
pop.simulations_dataframe = pd.concat(list_rows_distance, ignore_index=True)

In [12]:
pop.simulations_dataframe[pop.simulations_dataframe["opt_snr_net"] >= 2]

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,intrinsic_index,reference_snr_index,cal_idx,opt_snr_H1,mf_snr_H1,opt_snr_L1,mf_snr_L1,opt_snr_V1,mf_snr_V1,opt_snr_net,mf_snr_net,iota,phi_1,phi_2


In [13]:
def step(row):
    if row["opt_snr_net"] >= 11:
        return 1
    return 0

In [14]:
# Timevol = tv.TimeVolume(cosmology=Planck18, simulation=pop, function_detection=step)

In [15]:
# Timevol.sensitive_volume_calc()